## 경사하강법

### 미분이란
- differentiation은 변수의 움직임에 따른 변화를 측정하는 도구로 최적화에서 제일 많이 사용되는 기법
- 미분은 함수 f 의 주어진점 (x, f(x))에서 접선의 기울기를 구한다.

한점에서 접선의 기울기를 알면 어느 방향에서 함수값이 증가 or 감소하는지 알 수 있다.
- 증가 시키고 싶다면 미분값을 더하고
- 감소시키고 싶다면 미분값을 뺀다.

미분값이 음수라면 x+f^(x) < x 이므로 
- 왼쪽으로 이동하면 함수값이 증가
- 오른쪽으로 이동하면 함수값이 감소 한다.  

미분값이 양수라면 x+f^(x) >> x 이므로 
- 오른쪽으로 이동하면 함수값이 증가
- 왼쪽으로 이동하면 함수값이 감소 한다.

미분 값을 더하면 경사상승법(gradient ascent)이며 함수의 극대값의 위치를 구할 때 사용  
__미분 값을 빼면 경사 하강법(gradient descent) 이며 함수의 극소값의 위치를 구할 때 사용__  
경사 상승법/ 하강법은 극값에 도달하면 움직임을 멈춘다.

### 경사 하강법 알고리즘

__Input 값__  
- gradient : 미분 계산 함수
- lr : 학습률
- eps : 학습 종료 조건
- init : 시작점  

var = init  
grad = gradient(var)  
while(abs(grad) > eps):  
    var -= lr * grad __이부분이 x-f^(x)를 계산하는 부분, lr은 학습률로 미분을 통해 업데이트 속도 조절__  
    grad = gradient(var) __종료조건 성립 시 까지 미문 값 업데이트__

In [30]:
import sympy as sym
from sympy.abc import x
import numpy as np

In [15]:
def func(val):
    fun = sym.poly(x**2 + 2*x + 3)
    return fun.subs(x, val), fun

In [23]:
def func_gradient(fun, val):
    _, function = fun(val)
    diff = sym.diff(function, x)
    return diff.subs(x, val), diff

In [27]:
def gradient_descent(fun, init_point, lr_rate = 1e-2, epsilon = 1e-5):
    cnt = 0
    val = init_point
    diff,_ = func_gradient(fun, init_point)
    while np.abs(diff) > epsilon:
        val -= lr_rate * diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
    print(f"함수:{fun(val)[1]}, 연산횟수: {cnt}, 최소점: {val}, {fun(val)[0]}")

In [28]:
gradient_descent(func, np.random.uniform(-2, 2))

함수:Poly(x**2 + 2*x + 3, x, domain='ZZ'), 연산횟수: 572, 최소점: -1.00000490024949, 2.00000000002401


### 변수가 벡터라면?
- 백터가 입력인 다변수 함수의 경우 편미문을 사용한다.
- 각 변수 별로 편미분을 계산한 gradient 벡터를 이용하여 경사하강/상승법에 사용할 수 있다.

In [31]:
from sympy.abc import x, y

sym.diff(sym.poly(x**2 + 2*x*y + 3) + sym.cos(x + 2*y), x)

C:\Anaconda3\lib\site-packages\sympy\polys\polytools.py:75: SymPyDeprecationWarning: 

Mixing Poly with non-polynomial expressions in binary operations has
been deprecated since SymPy 1.6. Use the as_expr or as_poly method to
convert types instead. See https://github.com/sympy/sympy/issues/18613
for more info.

  SymPyDeprecationWarning(


2*x + 2*y - sin(x + 2*y)

### 그레디언트 벡터

https://towardsdatascience.com/a-visual-explanation-of-gradient-descent-methods-momentum-adagrad-rmsprop-adam-f898b102325c

- 3차원 공간상 함수 f(x,y)의 편미분 벡터 f는 f(x,y)함수의 최대값으로 향하는 화살표로 볼 수 있다.
- 변미분 벡터 f에 '-'를 붙이면  f(x,y) 함수에 최소점으로 향하는 화살표라고 볼 수 있다.
- 몇 차원이든 그레디언트 벡터를 이용하면 주어진 함수의 최소값, 최대값을 찾을 수 있고 경사 하강법, 상승법으로 최적화를 할 수 있다.

### 그레디언트 벡터 경사하강법 : 알고리즘
__input__  
- gradient : 그레디언트 벡터를 계산하는 함수
- init : 시작점, lr : 학습률, eps : 알고리즘 종료 조건  

__경사하강법 알고리즘은 그대로 적용되나 벡터는 절대값 대신 norm을 계산하여 종료조건 명시__  
var = init  
grad = gradient(var)  
while (norm(grad) > eps):  
    var -= lr * grad  
    grad = gradient(var)  

In [36]:
def eval_(fun, val):
    val_x, val_y = val
    fun_eval = fun.subs(x, val_x).subs(y, val_y)
    return fun_eval

In [41]:
def func_multi(val):
    x_,y_=val
    func = sym.poly(x**2 + 2*y**2)
    return eval_(func, [x_, y_]), func

In [47]:
def func_gradient(fun, val):
    x_, y_=val
    _, function = fun(val)
    diff_x = sym.diff(function, x)
    diff_y = sym.diff(function, y)
    grad_vec = np.array([eval_(diff_x, [x_, y_]), eval_(diff_y, [x_, y_])], dtype = float)
    return grad_vec, [diff_x, diff_y]

In [54]:
def gradient_descent(fun, init_point, lr_rate = 1e-2, epsilon = 1e-5):
    cnt = 0
    val = init_point
    diff,_ = func_gradient(fun, val)
    while np.linalg.norm(diff) > epsilon:
        val -= lr_rate*diff
        diff,_=func_gradient(fun, val)
        cnt += 1
    print(f"함수: {fun(val)[1]}, 연산횟수:{cnt}, 최소점: {val}, {fun(val)[0]}")

In [55]:
pt = [np.random.uniform(-2, 2), np.random.uniform(-2, 2)]
gradient_descent(func_multi, pt)

함수: Poly(x**2 + 2*y**2, x, y, domain='ZZ'), 연산횟수:603, 최소점: [4.91495332e-06 3.21118043e-11], 2.41567661439022E-11
